# Tricky Stuff

This file highlights some tricky aspects of Julia (from the perspective of a Matlab user).

## Load Packages and Extra Functions

In [1]:
using Printf, LinearAlgebra

include("src/printmat.jl");

# An Nx1 Matrix Is Not a Vector

and it sometimes matters.

Julia has both vectors and Nx1 matrices (the latter being a special case of NxM matrices). They can often be used interchangeably, but not always.

In [2]:
v  = ones(Int,2)                     #a vector with two elements
v2 = ones(Int,2,1)                   #a 2x1 matrix (Array)

println("v and v2 'look' similar:")
printmat(v)
printmat(v2)
println("but they have different sizes: ",size(v)," ",size(v2))

v and v2 'look' similar:
     1    
     1    

     1    
     1    

but they have different sizes: (2,) (2, 1)


# `X[1,:]` Gives a (Flat) Vector

If `X` is a $T\times n$ matrix, then `X[1,:]` gives a flat vector, *not* a $1 \times n$ matrix (or row vector).

In [3]:
X = [11 12;21 22]

x1 = X[1,:]                      #this gives a flat vector
println("size of X[1,:]: ", size(x1))
printmat(x1)

x1b = X[1:1,:]                   #however, this gives a 1x2 matrix
println("size of X[1:1,:]: ", size(x1b))
printmat(x1b)

size of X[1,:]: (2,)
    11    
    12    

size of X[1:1,:]: (1, 2)
    11        12    



# Array .+ Scalar Requires a Dot (.)

In [4]:
y = [1;2] .+ 1              #do not forget the dot (.)
printmat(y)

     2    
     3    



# Creating Variables in a Loop

In [5]:
for i = 1:5
    global Tor         #without this, Tor is not seen outside the loop
    Tor = cos(i)
end
println("Tor: $Tor")


Oden = Inf              #define `Oden` before the loop
for i = 1:5
    #global Oden        #only needed in script
    Oden = sin(i)       #will overwrite an existing value 
end
println("Oden: $Oden")

Tor: 0.28366218546322625
Oden: -0.9589242746631385


# An Array of Arrays

can be initialized by comprehension (see below). 

Do **not** use `fill(array,n)` for this, since all `n` arrays will refer to the same underlying array (changing one changes all).

In [6]:
x = [zeros(2,2) for i=1:2]        #a vector of two matrices
x[1][1,1] = -99                   #change an element of x[1]

println("x[1]")
printmat(x[1])

println("x[2]")
printmat(x[2])

x[1]
   -99.000     0.000
     0.000     0.000

x[2]
     0.000     0.000
     0.000     0.000



# Arrays are Different...

Vectors and matrices (arrays) can take lots of memory space, so **Julia is designed to avoid unnecessary copies of arrays**. Notice the following:
* ```B = A``` creates two names of the *same* array (changing one changes the other)
* ```B = reshape(A,n,m)```, ```B = vec(A)```, and  ```B = A'``` and create *another view* of the same array (changing one changes the other)
* `E = [C,D]` (where `C` and `D` are arrays) refers to underlying arrays. Changing `C` or `D` will affect `E` and vice versa.
* When an input an array to a function, then this array is shared between the function and the calling program (scope). Changing *elements* of the array (inside the function) will then change the array outside the function.

If you do not like this behaviour, then use `copy(A)` to create an independent copy of the array.

See the chapter on Arrays for more details.